In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('deliveries.csv')
pd.set_option('display.max_columns', None)

In [3]:
matches = pd.read_csv('matches.csv')
pd.set_option('display.max_columns', None)

In [4]:
df

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,2,0,0,0,0,0,2,2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179073,11415,2,Chennai Super Kings,Mumbai Indians,20,2,RA Jadeja,SR Watson,SL Malinga,0,0,0,0,0,0,1,0,1,NaN,NaN,NaN
179074,11415,2,Chennai Super Kings,Mumbai Indians,20,3,SR Watson,RA Jadeja,SL Malinga,0,0,0,0,0,0,2,0,2,NaN,NaN,NaN
179075,11415,2,Chennai Super Kings,Mumbai Indians,20,4,SR Watson,RA Jadeja,SL Malinga,0,0,0,0,0,0,1,0,1,SR Watson,run out,KH Pandya
179076,11415,2,Chennai Super Kings,Mumbai Indians,20,5,SN Thakur,RA Jadeja,SL Malinga,0,0,0,0,0,0,2,0,2,NaN,NaN,NaN


# DELIVERY WITHOUT SUPER OVER

In [5]:
delivery = df.drop(df[df['is_super_over'] != 0].index) 

# DELIVERIES WITHOUT SUPER OVER, WIDE RUNS, NO BALL RUNS

In [6]:
deliveries = df.drop(df[df['is_super_over'] != 0].index) 

In [7]:
deliveries = df.drop(df[df['wide_runs'] != 0].index) 

In [8]:
deliveries = df.drop(df[df['noball_runs'] != 0].index) 

# BOWLER INNINGS PLAYED => CALCULATE USING DELIVERY

In [9]:
bowler_innings_played=delivery.groupby(['bowler'])['match_id'].nunique().reset_index()

In [10]:
bowler_innings_played.rename(columns = {'match_id':'total_innings'}, inplace = True)

In [11]:
bowler_innings_played=bowler_innings_played[bowler_innings_played['total_innings']>20]

In [12]:
bowler_innings_played=bowler_innings_played.reset_index(drop=True)

In [13]:
bowler_names=bowler_innings_played['bowler'].tolist()

# RUNS => CALCULATE USING DELIVERY

In [14]:
bowler_runs=delivery.groupby(['bowler'])['total_runs'].sum().reset_index()

In [15]:
bowler_runs = bowler_runs[bowler_runs.bowler.isin(bowler_names)]

# NUM_BALLS => CALCULATE USING DELIVERIES


In [16]:
num_balls=deliveries['bowler'].value_counts().reset_index()

In [17]:
num_balls.rename(columns = {'index':'bowler', 'bowler':'total_balls'}, inplace = True)

In [18]:
num_balls = num_balls[num_balls.bowler.isin(bowler_names)]

# WIC_TAKEN_OVERALL => CALCULATE USING DELIVERY + RUN_OUT REMOVED

In [19]:
out = delivery
out['player_dismissed'] = out['player_dismissed'].replace(np.nan, 0)
out.player_dismissed[out.player_dismissed != 0] = 1

/var/folders/80/x1svf9ms1zvcvb7klmk6nzlr0000gn/T/ipykernel_79985/561585546.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out.player_dismissed[out.player_dismissed != 0] = 1


In [20]:
out = out.drop(out[out['dismissal_kind']=='run out'].index)

In [21]:
out = out[out.bowler.isin(bowler_names)]

In [22]:
wic_taken_overall = out.groupby(['bowler'])['player_dismissed'].sum().reset_index()

In [23]:
wic_taken_overall.rename(columns={'player_dismissed':'wic_taken'},inplace=True)

In [24]:
wic_taken_overall = wic_taken_overall[wic_taken_overall.bowler.isin(bowler_names)]

# BOWLER_RUN_CONCIDED USING DELIVERY => SUMING(WIDE_RUNS + NOBALL_RUNBS + BATSMAN_RUNS)


In [25]:
bowler_run_concided=delivery.groupby(['bowler'])['wide_runs','noball_runs','batsman_runs'].sum().reset_index()

/var/folders/80/x1svf9ms1zvcvb7klmk6nzlr0000gn/T/ipykernel_79985/784128777.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  bowler_run_concided=delivery.groupby(['bowler'])['wide_runs','noball_runs','batsman_runs'].sum().reset_index()


In [26]:
bowler_run_concided = bowler_run_concided.eval('Sum = wide_runs + noball_runs + batsman_runs')

In [27]:
bowler_run_concided=bowler_run_concided.drop(['wide_runs','noball_runs','batsman_runs'],axis=1)

In [28]:
bowler_run_concided.rename(columns={'Sum':'run_conceded'},inplace=True)

In [29]:
bowler_run_concided = bowler_run_concided[bowler_run_concided.bowler.isin(bowler_names)]

# WICKET_TAKEN_PER_MATCH USING DELIVERY

In [30]:
wicket_taken_per_match=out.groupby(['bowler','match_id'])['player_dismissed'].sum().reset_index()

In [31]:
wicket_taken_per_match = wicket_taken_per_match[wicket_taken_per_match.bowler.isin(bowler_names)]

# * NUM_TIMES_CRITICAL_WICKET_TAKEN (WICKET_TAKEN_PER_MATCH >=4 THEN VALUE COUNTS BOWLER)


In [32]:
num_times_critical_wicket_taken = wicket_taken_per_match[wicket_taken_per_match['player_dismissed']>=4]

In [33]:
num_times_critical_wicket_taken = num_times_critical_wicket_taken['bowler'].value_counts().reset_index()

In [34]:
num_times_critical_wicket_taken.rename(columns={'index':'bowler','bowler':'num_times_critical_wicket_taken'},inplace=True)

In [35]:
num_times_critical_wicket_taken

,bowler,num_times_critical_wicket_taken
0,SL Malinga,7
1,SP Narine,7
2,RA Jadeja,4
3,A Mishra,4
4,L Balaji,4
...,...,...
57,M Morkel,1
58,MA Starc,1
59,MM Sharma,1
60,MP Stoinis,1


# Final

In [36]:
final=pd.merge(bowler_innings_played,bowler_runs,on=['bowler'])

In [37]:
final=pd.merge(num_balls,final,on=['bowler'])

In [38]:
final=pd.merge(wic_taken_overall,final,on=['bowler'])

In [39]:
final=pd.merge(bowler_run_concided,final,on=['bowler'])

In [40]:
final=final.merge(num_times_critical_wicket_taken, on='bowler', how='left')

In [41]:
final

,bowler,run_conceded,wic_taken,total_balls,total_innings,total_runs,num_times_critical_wicket_taken
0,A Kumble,1058,45,978,42,1089,3.0
1,A Mishra,3804,156,3152,147,3850,4.0
2,A Nehra,2495,106,1965,88,2537,1.0
3,A Singh,742,31,550,26,769,1.0
4,A Symonds,674,20,539,30,694,NaN
...,...,...,...,...,...,...,...
120,WD Parnell,701,27,596,26,731,NaN
121,YK Pathan,1415,42,1184,82,1443,NaN
122,YS Chahal,2366,100,1835,83,2416,2.0
123,Yuvraj Singh,1078,36,881,73,1092,2.0


# METRICS_RESULTS 

In [42]:
metrics_final=final['bowler'].reset_index()

# 1

In [43]:
metrics_final=metrics_final.drop(columns={'index'},axis=1)

In [44]:
metrics_final['economy'] = final['total_runs'] / (final['total_balls']/6).round(decimals=2)

In [45]:
metrics_final['economy'] = metrics_final['economy'].round(decimals=2)

# 2

In [46]:
metrics_final['wic_tak_ability'] = final['total_balls'] / final['wic_taken']

In [47]:
metrics_final['wic_tak_ability'] = metrics_final['wic_tak_ability'].astype(float)

In [48]:
metrics_final['wic_tak_ability'] = metrics_final['wic_tak_ability'].round(decimals=2)

# 3

In [49]:
metrics_final['concistency'] = final['run_conceded'] / final['wic_taken']

In [50]:
metrics_final['concistency'] = metrics_final['concistency'].astype(float)

In [51]:
metrics_final['concistency'] = metrics_final['concistency'].round(decimals=2)

# 4

In [52]:
metrics_final['num_times_critical_wicket_taken'] = final['num_times_critical_wicket_taken'] 

In [53]:
metrics_final['num_times_critical_wicket_taken'] = metrics_final['num_times_critical_wicket_taken'].replace(np.nan,0)

In [54]:
metrics_final['num_times_critical_wicket_taken'] = metrics_final['num_times_critical_wicket_taken'].astype(int)

In [55]:
metrics_final.to_csv('bowler_metrics_final.csv')

In [56]:
metrics_final

,bowler,economy,wic_tak_ability,concistency,num_times_critical_wicket_taken
0,A Kumble,6.68,21.73,23.51,3
1,A Mishra,7.33,20.21,24.38,4
2,A Nehra,7.75,18.54,23.54,1
3,A Singh,8.39,17.74,23.94,1
4,A Symonds,7.73,26.95,33.70,0
...,...,...,...,...,...
120,WD Parnell,7.36,22.07,25.96,0
121,YK Pathan,7.31,28.19,33.69,0
122,YS Chahal,7.90,18.35,23.66,2
123,Yuvraj Singh,7.44,24.47,29.94,2


# Graphs_df

In [57]:
season=matches[['id','season']]

In [58]:
season.rename(columns={'id':'match_id'},inplace=True)

/var/folders/80/x1svf9ms1zvcvb7klmk6nzlr0000gn/T/ipykernel_79985/1305305918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season.rename(columns={'id':'match_id'},inplace=True)


# RUNS_PER_YEAR => USING DELIVERY

In [59]:
run_per_year = delivery[delivery.bowler.isin(bowler_names)]

In [60]:
run_per_year=run_per_year.groupby(['match_id','bowler'])['total_runs'].sum().reset_index()

In [61]:
run_per_year=pd.merge(run_per_year,season,on=['match_id'])

In [62]:
run_per_year=run_per_year.groupby(['bowler','season'])['total_runs'].sum().reset_index()

# BALLS_BOWLED_PER_YEAR => USING DELIVERIES


In [63]:
balls_bowled_per_year = deliveries[deliveries.bowler.isin(bowler_names)]

In [64]:
balls_bowled_per_year = balls_bowled_per_year.groupby(['match_id', 'bowler'])['ball'].count().reset_index()

In [65]:
balls_bowled_per_year = pd.merge(balls_bowled_per_year,season,on=['match_id'])

In [66]:
balls_bowled_per_year=balls_bowled_per_year.groupby(['bowler','season'])['ball'].sum().reset_index()

# YEARLY_WICK_TAKEN => USING DELIVERY + REMOVING RUN_OUT


In [67]:
yearly_wicket_taken = out.groupby(['match_id','bowler'])['player_dismissed'].sum().reset_index()

In [68]:
yearly_wicket_taken = pd.merge(yearly_wicket_taken,season,on=['match_id'])

In [69]:
yearly_wicket_taken=yearly_wicket_taken.groupby(['bowler','season'])['player_dismissed'].sum().reset_index()

# BOWLER_RUN_CONCIDED_YEARLY => SUMING(WIDE_RUNS + NOBALL_RUNBS + BATSMAN_RUNS)

In [70]:
bowler_run_concided_yearly=delivery[delivery.bowler.isin(bowler_names)]

In [71]:
bowler_run_concided_yearly=bowler_run_concided_yearly.groupby(['match_id','bowler'])['wide_runs','noball_runs','batsman_runs'].sum().reset_index()

/var/folders/80/x1svf9ms1zvcvb7klmk6nzlr0000gn/T/ipykernel_79985/3441594736.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  bowler_run_concided_yearly=bowler_run_concided_yearly.groupby(['match_id','bowler'])['wide_runs','noball_runs','batsman_runs'].sum().reset_index()


In [72]:
bowler_run_concided_yearly = bowler_run_concided_yearly.eval('run_concided = wide_runs + noball_runs + batsman_runs')

In [73]:
bowler_run_concided_yearly = bowler_run_concided_yearly[['match_id','bowler','run_concided']]

In [74]:
bowler_run_concided_yearly = pd.merge(bowler_run_concided_yearly,season,on=['match_id'])

In [75]:
bowler_run_concided_yearly=bowler_run_concided_yearly.groupby(['bowler','season'])['run_concided'].sum().reset_index()

# CRITICAL_WIC_TAKEN_YEARLY

In [76]:
critical_wicket_taken_yearly = pd.merge(wicket_taken_per_match,season,on='match_id')

In [77]:
critical_wicket_taken_yearly = critical_wicket_taken_yearly.groupby(['bowler','season'])['player_dismissed'].sum().reset_index()

In [78]:
critical_wicket_taken_yearly

,bowler,season,player_dismissed
0,A Kumble,2008,7
1,A Kumble,2009,21
2,A Kumble,2010,17
3,A Mishra,2008,11
4,A Mishra,2009,14
...,...,...,...
808,Z Khan,2013,5
809,Z Khan,2014,5
810,Z Khan,2015,7
811,Z Khan,2016,10


# FINAL GRAPH DF

In [79]:
final_graph=run_per_year

In [80]:
final_graph['balls_bowled']=balls_bowled_per_year['ball']

In [81]:
final_graph['wic_taken']=yearly_wicket_taken['player_dismissed']

In [82]:
final_graph['run_concided']=bowler_run_concided_yearly['run_concided']

In [83]:
final_graph.to_csv('bowler_final_graph.csv')

In [84]:
final_graph['economy']=final_graph['total_runs'] / (final_graph['balls_bowled']/6).round(decimals=2)

In [85]:
final_graph['concistency']=final_graph.run_concided[final_graph['wic_taken']!=0] / final_graph.wic_taken[final_graph['wic_taken']!=0]

In [86]:
final_graph['concistency']=final_graph['concistency'].astype(float)

In [87]:
final_graph['economy']=final_graph['economy'].astype(float)

In [88]:
final_graph['concistency']=final_graph['concistency'].round(decimals=2)

In [89]:
final_graph['economy']=final_graph['economy'].round(decimals=2)

In [90]:
final_graph['critical_wicket_taken']=critical_wicket_taken_yearly['player_dismissed']

In [91]:
final_graph

,bowler,season,total_runs,balls_bowled,wic_taken,run_concided,economy,concistency,critical_wicket_taken
0,A Kumble,2008,314,236,7,304,7.98,43.43,7
1,A Kumble,2009,353,358,21,347,5.92,16.52,21
2,A Kumble,2010,422,384,17,407,6.59,23.94,17
3,A Mishra,2008,140,122,11,138,6.89,12.55,11
4,A Mishra,2009,300,256,14,294,7.03,21.00,14
...,...,...,...,...,...,...,...,...,...
808,Z Khan,2013,47,38,5,47,7.42,9.40,5
809,Z Khan,2014,156,138,5,146,6.78,29.20,5
810,Z Khan,2015,159,155,7,156,6.16,22.29,7
811,Z Khan,2016,341,271,10,337,7.55,33.70,10


In [92]:
final_graph.to_csv('bowler_final_graph.csv')